In [1]:
# Install required packages
!pip install -q streamlit scikit-learn pandas joblib pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
import os

In [3]:
# Load dataset (Glass Identification UCI)
from sklearn.datasets import load_iris  # Used as placeholder, replace with actual glass dataset if available


In [4]:
# Simulated placeholder dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data"
df = pd.read_csv(url, header=None)
df.columns = ['Id', 'RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type']
df.drop('Id', axis=1, inplace=True)

In [5]:
# Preprocessing
X = df.drop("Type", axis=1)
y = df["Type"]


In [6]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [7]:
# Pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model', RandomForestClassifier(random_state=42))])

In [8]:
# Save the model
joblib.dump(pipe, 'glass_model.pkl')
print("✅ Model trained and saved as glass_model.pkl")


✅ Model trained and saved as glass_model.pkl


In [ ]:
# Save the Streamlit app code
with open("app.py", "w") as f:
    f.write('''
import streamlit as st
import pandas as pd
import joblib

st.set_page_config(page_title="Glass Type Prediction", layout="centered")

model = joblib.load("glass_model.pkl")

glass_dict = {
    1: 'Building Windows (Float Process)',
    2: 'Building Windows (Non Float Process)',
    3: 'Vehicle Windows',
    5: 'Containers',
    6: 'Tableware',
    7: 'Headlamps'
}

st.title("🔍 Glass Type Prediction")
st.markdown("Enter the chemical composition of a glass sample to predict its type.")

def user_input_features():
    RI = st.slider("Refractive Index (RI)", 1.5, 1.6, 1.52)
    Na = st.slider("Sodium (Na)", 10.0, 17.0, 13.0)
    Mg = st.slider("Magnesium (Mg)", 0.0, 5.0, 2.0)
    Al = st.slider("Aluminum (Al)", 0.0, 5.0, 1.5)
    Si = st.slider("Silicon (Si)", 69.0, 75.0, 72.0)
    K  = st.slider("Potassium (K)", 0.0, 6.0, 1.0)
    Ca = st.slider("Calcium (Ca)", 5.0, 16.0, 8.0)
    Ba = st.slider("Barium (Ba)", 0.0, 4.0, 0.1)
    Fe = st.slider("Iron (Fe)", 0.0, 1.0, 0.1)

    data = {
        "RI": RI, "Na": Na, "Mg": Mg, "Al": Al, "Si": Si,
        "K": K, "Ca": Ca, "Ba": Ba, "Fe": Fe
    }
    return pd.DataFrame(data, index=[0])

input_df = user_input_features()

if st.button("Predict Glass Type"):
    prediction = model.predict(input_df)[0]
    proba = model.predict_proba(input_df)[0]
    st.success(f"🔬 Predicted Type: **{glass_dict.get(prediction, 'Unknown')}** (Class {prediction})")

    st.subheader("📊 Prediction Probabilities")
    prob_df = pd.DataFrame(proba, index=[f"Class {i+1}" for i in range(len(proba))], columns=["Probability"])
    st.bar_chart(prob_df)

if hasattr(model.named_steps['model'], 'feature_importances_'):
    st.subheader("📈 Feature Importance")
    feat_df = pd.DataFrame({
        "Feature": input_df.columns,
        "Importance": model.named_steps['model'].feature_importances_
    }).sort_values(by="Importance", ascending=False)
    st.write(feat_df)
    st.bar_chart(feat_df.set_index("Feature"))

st.markdown("---")
st.markdown("© 2025 Emmanuel Osei | [GitHub](https://github.com)")
''')

print("✅ Streamlit app written to app.py")

from pyngrok import ngrok

# Kill any existing tunnels
ngrok.kill()

# Run Streamlit app
!streamlit run app.py &

# Expose the port 8501
public_url = ngrok.connect(port='8501')
print(f"🌐 Your app is live at: {public_url}")

✅ Streamlit app written to app.py



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.75.205.44:8501

